
















Import packages

In [ ]:
from tensorflow.keras.layers import Conv1D, Bidirectional, LSTM, Dense, Input, Dropout
from tensorflow.keras.layers import SpatialDropout1D
from tensorflow.keras.callbacks import ModelCheckpoint
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import itertools
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, f1_score
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from nltk.tokenize import word_tokenize
from nltk import sent_tokenize
nltk.download('punkt')
import joblib
from keras.preprocessing.text import Tokenizer
import gensim
from sklearn.preprocessing import LabelEncoder
from keras.layers import Embedding
from keras.models import Sequential
from keras.layers import Dense,LSTM,Dropout
from sklearn.svm import SVC
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import warnings
warnings.filterwarnings('ignore')
import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
pip install --upgrade keras

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Obtain raw data

In [ ]:
df = pd.read_csv('https://git.wayne.edu/gh7553/twitter_dataset_sentiment-analysis/-/raw/master/training.1600000.processed.noemoticon.csv',names=['target','ids','date','flag','user','raw_text'])

Some EDA

In [ ]:
df.head()

,target,ids,date,flag,user,raw_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [ ]:
df.isna().sum()


target      0
ids         0
date        0
flag        0
user        0
raw_text    0
dtype: int64

In [ ]:
df.shape


(1600000, 6)

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1600000 entries, 0 to 1599999
Data columns (total 6 columns):
 #   Column    Non-Null Count    Dtype 
---  ------    --------------    ----- 
 0   target    1600000 non-null  int64 
 1   ids       1600000 non-null  int64 
 2   date      1600000 non-null  object
 3   flag      1600000 non-null  object
 4   user      1600000 non-null  object
 5   raw_text  1600000 non-null  object
dtypes: int64(2), object(4)
memory usage: 73.2+ MB




---

preprocessing steps for data

1. remove links
2. remove @'s
3. lower-case all text
4. remove puncuation
5. remove stopwords
6. tokenize words per row
7. stem words per row

---





In [ ]:
df['process_text'] = df['raw_text']

In [ ]:
#Remove Links and handles from the raw tweet data using regex
def remove_links_and_handles(text):
    pattern_links = r"^[-a-zA-Z0-9@:%._\\+~#=]{1,256}\\.[a-zA-Z0-9()]{1,6}\\b(?:[-a-zA-Z0-9()@:%_\\+.~#?&\\/=]*)$"
    text = re.sub(pattern_links, '', text)
    pattern_handles = r'@[^\s]+'
    text = re.sub(pattern_handles, '', text)
    return text

In [ ]:
#remove puncuation from raw tweet data using regex
def remove_punctuation(text):
    pattern = r'[^\w\s]'
    text = re.sub(pattern, '', text)
    return text

In [ ]:
#applying removing link and puncuation with making everything lowercase
df['process_text'] = df['process_text'].apply(remove_links_and_handles)
df['process_text'] = df['process_text'].apply(remove_punctuation)
df['process_text'] = df['process_text'].str.lower()

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [ ]:
#tokenize the tweets
df['process_text'] = df['process_text'].apply(word_tokenize)

In [ ]:
#remove stopwords that do not add any value in evaluation of sentiment for a tweet
def remove_stopwords(tokens):
    stop_words = set(stopwords.words('english'))
    filtered_tokens = [token for token in tokens if token.lower() not in stop_words]
    return filtered_tokens

In [ ]:
df['process_text'] = df['process_text'].apply(remove_stopwords)

In [ ]:
df['process_text']

0          [httptwitpiccom2y1zl, awww, thats, bummer, sho...
1          [upset, cant, update, facebook, texting, might...
2          [dived, many, times, ball, managed, save, 50, ...
3                    [whole, body, feels, itchy, like, fire]
4                             [behaving, im, mad, cant, see]
                                 ...                        
1599995                  [woke, school, best, feeling, ever]
1599996    [thewdbcom, cool, hear, old, walt, interviews,...
1599997                [ready, mojo, makeover, ask, details]
1599998    [happy, 38th, birthday, boo, alll, time, tupac...
1599999                              [happy, charitytuesday]
Name: process_text, Length: 1600000, dtype: object

In [ ]:
df.head()

,target,ids,date,flag,user,raw_text,process_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","[httptwitpiccom2y1zl, awww, thats, bummer, sho..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"[upset, cant, update, facebook, texting, might..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"[dived, many, times, ball, managed, save, 50, ..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"[whole, body, feels, itchy, like, fire]"
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","[behaving, im, mad, cant, see]"


In [ ]:
df.to_csv(r'./pre_processed_data.csv', index=False)

In [ ]:
#easy access to new data post processing
pre_processed_data = pd.read_csv('https://git.wayne.edu/gh7553/twitter_dataset_sentiment-analysis/-/raw/master/pre_processed_data.csv')

In [ ]:
pre_processed_data.head()

,target,ids,date,flag,user,raw_text,process_text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...","['httptwitpiccom2y1zl', 'awww', 'thats', 'bumm..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,"['upset', 'cant', 'update', 'facebook', 'texti..."
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,"['dived', 'many', 'times', 'ball', 'managed', ..."
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,"['whole', 'body', 'feels', 'itchy', 'like', 'f..."
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....","['behaving', 'im', 'mad', 'cant', 'see']"


In [ ]:
#Change the binary "target" values to (0,1)

pre_processed_data['target'] = pre_processed_data['target'].replace(4,1)

pre_processed_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

EDA

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

eda_df = pre_processed_data
eda_df.columns
sns.countplot(data = eda_df, x = 'target')
plt.title("Count Plot of target class")
plt.show()

> There appears to be no class imbalance

In [ ]:
eda_df.head()
eda_df['target'].value_counts()


Word Cloud Generation

In [ ]:
#creating a list with postive words for word cloud
positive_list= []
positive = eda_df[eda_df['target'] == 1]
positive_list.append(positive['process_text'])

negative_list = []
negative = eda_df[eda_df['target'] == 0]
negative_list.append(negative['process_text'])

# Joining all words in the list
positive_all = " ".join([word for sent in positive_list for word in sent ])
negative_all = " ".join([word for sent in negative_list for word in sent ])

In [ ]:
# Importing Word Cloud Package
from wordcloud import WordCloud

#Creating a word cloud object for postive words
wordcloud = WordCloud(width=2000,height=1000,background_color='#000000',max_words = 500).generate(positive_all)

#plotting Postive word cloud
plt.figure(figsize=(20,30))
plt.imshow(wordcloud)
plt.title("Recursive Positive Words")
plt.show()


In [ ]:
#Creating word cloud object for negative words
wordcloud = WordCloud(width=2000,height=1000,background_color='#000000',max_words = 500).generate(negative_all)

#Plotting negative word cloud
plt.figure(figsize=(20,30))
plt.imshow(wordcloud)
plt.title("Recursive Negative Words")
plt.show()


Logistic Regression Model

In [ ]:
# Split Test/Train Data

X_train, X_test, y_train, y_test = train_test_split(pre_processed_data["process_text"], pre_processed_data["target"], test_size=0.3, random_state=39)

In [ ]:
# Vectorize Data

vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# LOGISTIC REGRESSION MODEL

lr = LogisticRegression()
lr.fit(X_train, y_train)
lr.score(X_test, y_test)


print("F1 Score: ", f1_score(y_test, lr.predict(X_test)))
print('Accuracy: ', accuracy_score(y_test, lr.predict(X_test)), "\n")
print(classification_report(y_test, lr.predict(X_test)))
print('Confusion Matrix:\n', confusion_matrix(y_test, lr.predict(X_test)))

In [ ]:
# hyperparameter tuning

from sklearn.model_selection import GridSearchCV

# Define the hyperparameters to be tuned
params = {'C': [0.001, 0.01, 0.1, 1, 10, 100],
          'penalty': ['l1', 'l2']}

# Use GridSearchCV to search for the best hyperparameters
grid_search = GridSearchCV(lr, params, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)

# Print the best hyperparameters and corresponding accuracy score
print("Best hyperparameters: ", grid_search.best_params_)
print("Best accuracy score: ", grid_search.best_score_)

# Use the best hyperparameters to make predictions on the test set
best_lr = grid_search.best_estimator_
print("F1 Score: ", f1_score(y_test, best_lr.predict(X_test)))
print('Accuracy: ', accuracy_score(y_test, best_lr.predict(X_test)), "\n")
print(classification_report(y_test, best_lr.predict(X_test)))
print('Confusion Matrix:\n', confusion_matrix(y_test, best_lr.predict(X_test)))


LSTM Models

In [ ]:
import pandas as pd


# convert the "process_text" column to a string with spaces
pre_processed_data['string_text'] = pre_processed_data['process_text'].apply(lambda x: ' '.join(x))

# print the updated dataframe
pre_processed_data.head()

In [ ]:
#splitting into testing/training
X_train, X_test, y_train, y_test = train_test_split(pre_processed_data["string_text"], pre_processed_data["target"], test_size=0.3, random_state=39)

In [ ]:
pre_processed_data.head()

In [ ]:
#tokenizing data
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X_train.values)
word_index = tokenizer.word_index
vocab_size = len(tokenizer.word_index) + 1
print("Vocabulary Size :", vocab_size)

In [ ]:
# Sequence padding on text
from tensorflow.keras.preprocessing.sequence import pad_sequences

x_train = pad_sequences(tokenizer.texts_to_sequences(X_train.values),
                        maxlen = 30)
x_test = pad_sequences(tokenizer.texts_to_sequences(X_test.values),
                       maxlen = 30)

print("Training X Shape:",x_train.shape)
print("Testing X Shape:",x_test.shape)

In [ ]:
# Getting y values in a readable shape
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

In [ ]:
#loading in GloVe embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip




--2023-04-29 06:36:50--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2023-04-29 06:36:50--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2023-04-29 06:36:50--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [ ]:

# Setting parameterings
GLOVE_EMB = 'glove.6B.300d.txt'
EMBEDDING_DIM = 300
LR = 1e-3
MODEL_PATH = 'best_model.hdf5'



In [ ]:
#loading embeddings
embeddings_index = {}

f = open(GLOVE_EMB)
for line in f:
  values = line.split()
  word = value = values[0]
  coefs = np.asarray(values[1:], dtype='float32')
  embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' %len(embeddings_index))



Found 400000 word vectors.


In [ ]:
# Creating embedding matrix
embedding_matrix = np.zeros((vocab_size, EMBEDDING_DIM))
for word, i in word_index.items():
  embedding_vector = embeddings_index.get(word)
  if embedding_vector is not None:
    embedding_matrix[i] = embedding_vector

In [ ]:
# Setting embedding layer for LSTM
embedding_layer = tf.keras.layers.Embedding(vocab_size,
                                          EMBEDDING_DIM,
                                          weights=[embedding_matrix],
                                          input_length=30,
                                          trainable=False)

In [ ]:
# Creating parameters for LSTM
# Define input shape & embeddings
sequence_input = Input(shape=(30,), dtype='int32')
embedding_sequences = embedding_layer(sequence_input)
#  Spatial Dropout layer
x = SpatialDropout1D(0.2)(embedding_sequences)
# dense layer with ReLU
x = Conv1D(64, 5, activation='relu')(x)
# Bidirection LSTM
x = Bidirectional(LSTM(64, dropout=0.2, recurrent_dropout=0.2))(x)
# Dense layer with 512 with ReLU
x = Dense(512, activation='relu')(x)
# Dropout layer to prevent overfitting
x = Dropout(0.5)(x)
# Another dense layer
x = Dense(512, activation='relu')(x)
# 1D Output layer
outputs = Dense(1, activation='sigmoid')(x)
#Defining model
model = tf.keras.Model(sequence_input, outputs)

In [ ]:
# Setting loss function and other NN parameters
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

model.compile(optimizer=Adam(learning_rate=LR), loss='binary_crossentropy',
              metrics=['accuracy'])
ReduceLROnPlateau = ReduceLROnPlateau(factor=0.1,
                                     min_lr = 0.01,
                                     monitor = 'val_loss',
                                     verbose = 1)

In [ ]:
#Training LSTM
history = model.fit(x_train, y_train, batch_size= 800, epochs=10,
                    validation_data=(x_test, y_test), callbacks=[ReduceLROnPlateau])


Epoch 1/10
1400/1400 [==============================] - 1344s 956ms/step - loss: 0.5890 - accuracy: 0.6764 - val_loss: 0.5548 - val_accuracy: 0.7047 - lr: 0.0010
Epoch 2/10
 710/1400 [==============>...............] - ETA: 10:07 - loss: 0.5565 - accuracy: 0.7076

KeyboardInterrupt: ignored

In [ ]:
# Plotting LSTM validation accuracy
s, (at, al) = plt.subplots(2,1)
at.plot(history.history['accuracy'], c= 'b')
at.plot(history.history['val_accuracy'], c='r')
at.set_title('model accuracy')
at.set_ylabel('accuracy')
at.set_xlabel('epoch')
at.legend(['LSTM_train', 'LSTM_val'], loc='upper left')

al.plot(history.history['loss'], c='m')
al.plot(history.history['val_loss'], c='c')
al.set_title('model loss')
al.set_ylabel('loss')
al.set_xlabel('epoch')
al.legend(['train', 'val'], loc = 'upper left')

In [ ]:
X_train

SVM

In [ ]:
# TRIM THE DATA TO 10% IN ORDER TO RUN CODE WITHOUT RUNNING OUT OF MEMORY AND COMPUTING POWER
df_sampled = pre_processed_data.sample(frac=0.1, random_state=42)

In [ ]:
# SPLIT DATA INTO TRAIN/TEST
X_train, X_test, y_train, y_test = train_test_split(df_sampled["process_text"], df_sampled["target"], test_size=0.3, random_state=39)

In [ ]:
# SVM MODEL
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC


# Define a custom tokenizer function
def my_tokenizer(text):
    tokens = text.lower().split()  # Split text into words
    return tokens

# Convert text data into numerical features using TF-IDF representation
vectorizer = TfidfVectorizer(tokenizer=my_tokenizer)

X_train_list = X_train.tolist()
X_test_list = X_test.tolist()

X_train_tfidf = vectorizer.fit_transform(X_train_list)
X_test_tfidf = vectorizer.transform(X_test_list)

svc_model = SVC()
svc_model.fit(X_train_tfidf, y_train)
y_pred = svc_model.predict(X_test_tfidf)

test_acc = accuracy_score(y_test, y_pred)
print(test_acc)

svc_cfm = confusion_matrix(y_test, y_pred)
print(svc_cfm)

HYPER-PARAMETER TUNERS

In [ ]:
#GRID HYPERPARAMETER SEARCH
param_grid = {'C': [0.1, 1, 10], 'kernel': ['linear', 'rbf']}

grid_search = GridSearchCV(svc_model, param_grid, cv=5)
grid_search.fit(X_train_tfidf, y_train)

best_params = grid_search.best_params_


In [ ]:
#RANDOM HYPERPARAMETER SEARCH
from scipy.stats import loguniform
from sklearn.model_selection import RandomizedSearchCV

param_distributions = {'C': loguniform(0.1, 1, 10), 'kernel': ['linear', 'rbf']}
random_search = RandomizedSearchCV(svc_model, param_distributions, n_iter=10, cv=5)
random_search.fit(X_train_tfidf, y_train)

best_params_random = random_search.best_params

In [ ]:
#BAYES HYPERPARAMETER SEARCH

from skopt import BayesSearchCV
from skopt.space import Real, Categorical, Integer

param_space = {'C': (0.1, 10.0, 'log-uniform'),
               'kernel': ['linear', 'rbf'],
               'gamma': (1e-6, 1e+1, 'log-uniform'),
               'degree': (1, 8, 'uniform')}

opt = BayesSearchCV(svc_model, param_space, n_iter=50, cv=5, n_jobs=-1)
opt.fit(X_train_tfidf, y_train)

best_params_bayes = opt.best_params_

best_svm = SVC(**best_params_bayes)
best_svm.fit(X_train_tfidf, y_train)

accuracy = best_svm.score(X_test_tfidf, y_test)
print("Best SVM :", best_params_bayes)
print("Accuracy :", accuracy)

Bag of Words/ Term Frequiency
-only count so does not capture the order in which the words go in

In [ ]:
x = pre_processed_data.process_text
y = pre_processed_data.target

vect = CountVectorizer()
vect.fit(x_train)
CountVectorizer()


vect_tunned = CountVectorizer(stop_words='english', ngram_range=(1,2), min_df=0.1, max_df=0.7, max_features=100)
vect_tunned
CountVectorizer(max_df=0.7, max_features=100, min_df=0.1, ngram_range=(1, 2),
                stop_words='english')

model = SVC(kernel='linear', random_state = 10)
model.fit(x_train_dtm, y_train)
#predicting output for test data
pred = model.predict(x_test_dtm)

Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

#Training Model
rfc=RandomForestClassifier(n_estimators=200,criterion="entropy")
rfc.fit(X_train,y_train)
y_pred=rfc.predict(X_test)

#Model Metrics
print(classification_report(y_test,y_pred))
print("The test accuracy score of Random Forest Classificatoin is: {:<.2f}% \n".format( accuracy_score(y_test, y_pred)*100))


Decision Tree

In [ ]:
#Installing Packages
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix,classification_report,accuracy_score,roc_auc_score
from sklearn import tree
from sklearn.tree import plot_tree

#Model Creation
clf = DecisionTreeClassifier(criterion="entropy")
clf.fit(X_train,y_train)
y_pred=clf.predict(X_test)

#Model Mertics
print(classification_report(y_test,y_pred))
print("The test accuracy score of Decision Tree is: {:<.2f}% \n".format( accuracy_score(y_test, y_pred)*100))

XG Boost

In [ ]:
#Importing Packages
import xgboost as xgb
from xgboost import XGBClassifier


#Model Creation
xg =XGBClassifier()
xg.fit(X_train,y_train)
y_pred= xg.predict(X_test)

#Model Metrics
print("The test accuracy score of XGBoost is: {:<.2f}% \n".format( accuracy_score(y_test, y_pred)*100))
print(classification_report(y_test,y_pred))
